# Day 01 — From Prompt to Action

This notebook introduces the foundational workflow for building AI agents with the Google ADK:

- Minimal ADK agent setup  
- In-memory runner  
- Constructing a user message  
- Streaming and extracting the final model response  
- Reproducing the Kaggle Day-1 notebook using a clean local environment  

This serves as the baseline for Day-2 (Tools & MCP) and Day-3 (Context Engineering).


## Environment Variables & Setup

In [1]:
# --- Load environment variables ---
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not set — please create a .env file.")

print("API key loaded.")


API key loaded.


## ADK + GenAI Imports

In [2]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.genai import types  # Generative content types (Content, Part)

print("ADK imports loaded successfully.")


ADK imports loaded successfully.


## Session Constants

In [3]:
# Constants for local testing
APP_NAME = "google_ai_agents_course"
USER_ID = "local_user_001"
SESSION_ID = "session_day01_001"

print(f"Using APP_NAME={APP_NAME}, SESSION_ID={SESSION_ID}")


Using APP_NAME=google_ai_agents_course, SESSION_ID=session_day01_001


## Define the Agent

In [4]:
agent = Agent(
    name="helpful_assistant",
    model="gemini-2.5-flash",
    instruction="You are a helpful assistant that explains concepts clearly."
)

print("Agent created:", agent.name)


Agent created: helpful_assistant


## Create the Runner

In [5]:
runner = InMemoryRunner(
    agent=agent,
    app_name=APP_NAME,
)

print("InMemoryRunner ready.")


InMemoryRunner ready.


## Create Helper Function for Single-Turn Execution

In [6]:
import asyncio

async def run_one_turn(question: str):
    """
    Executes a single request through the ADK agent using session state.
    Returns the model's final text response.
    """

    # --- Ensure session exists ---
    await runner.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
    )

    # --- Convert user input into generative Content ---
    user_message = types.Content(
        role="user",
        parts=[types.Part(text=question)]
    )

    # --- Iterate over streaming events ---
    final_text = None

    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=user_message,
    ):

        # Capture the final model output
        if event.is_final_response() and event.content:
            final_text = "".join(
                (part.text or "")
                for part in event.content.parts
                if getattr(part, "text", None)
            )

    return final_text


## First Test Run

In [7]:
answer = await run_one_turn("What is an AI agent?")
print(answer)

An **AI agent** is essentially an artificial intelligence system that can **perceive its environment**, make **decisions** based on those perceptions and its internal state, and then take **actions** to achieve specific **goals**.

Think of it like this:

*   **You** are an agent in the real world. You perceive (see, hear, feel), decide what to do (get a coffee), and act (walk to the kitchen, make coffee) to achieve your goal (caffeine!).
*   An **AI agent** does the same thing, but in its defined digital or physical domain.

Here's a breakdown of the key components:

1.  **Perception:**
    *   How the agent "sees" or "understands" its environment.
    *   This could be through sensors (cameras, microphones, lidar for a robot), data input (text, numbers, database entries for a software agent), or user queries.
    *   *Example:* A self-driving car perceives its surroundings using cameras, radar, and GPS. A chatbot perceives user input as text.

2.  **Environment:**
    *   The world o